In [1]:
from winners_only import *
from log import *

import pandas as pd
import numpy as np
import math

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import log_loss, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# Import Data

In [2]:
df = pd.read_csv("data/updated_data.csv")

## Clean and split

In [3]:
X,y = winners_only_cleaning(df)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state = 0)

In [12]:
X.head()

,no_of_rounds,avg_BODY_att,avg_BODY_landed,avg_CLINCH_att,avg_CLINCH_landed,avg_DISTANCE_att,avg_DISTANCE_landed,avg_GROUND_att,avg_GROUND_landed,avg_HEAD_att,avg_HEAD_landed,avg_KD,avg_LEG_att,avg_LEG_landed,avg_PASS,avg_REV,avg_SIG_STR_att,avg_SIG_STR_landed,avg_SIG_STR_pct,avg_SUB_ATT,avg_TD_att,avg_TD_landed,avg_TD_pct,avg_TOTAL_STR_att,avg_TOTAL_STR_landed,avg_opp_BODY_att,avg_opp_BODY_landed,avg_opp_CLINCH_att,avg_opp_CLINCH_landed,avg_opp_DISTANCE_att,avg_opp_DISTANCE_landed,avg_opp_GROUND_att,avg_opp_GROUND_landed,avg_opp_HEAD_att,avg_opp_HEAD_landed,avg_opp_KD,avg_opp_LEG_att,avg_opp_LEG_landed,avg_opp_PASS,avg_opp_REV,avg_opp_SIG_STR_att,avg_opp_SIG_STR_landed,avg_opp_SIG_STR_pct,avg_opp_SUB_ATT,avg_opp_TD_att,avg_opp_TD_landed,avg_opp_TD_pct,avg_opp_TOTAL_STR_att,avg_opp_TOTAL_STR_landed,total_rounds_fought,total_time_fought(seconds),Height_cms,Reach_cms,Weight_lbs,age
0,5,21.900000,16.400000,17.000000,11.000000,75.000000,26.500000,9.400000,6.500000,74.200000,23.900000,0.4,5.300000,3.700000,1.200000,0.000000,101.400000,44.000000,0.466000,0.100000,5.300000,1.900000,0.458000,129.900000,69.100000,13.300000,8.800000,7.500000,5.100000,90.500000,26.800000,0.800000,0.300000,76.100000,17.300000,0.100000,9.400000,6.100000,0.000000,0.000000,98.800000,32.200000,0.336000,0.000000,0.900000,0.100000,0.050000,110.500000,43.300000,27.0,742.600000,162.56,162.56,135.0,32.0
1,5,12.000000,7.714286,9.285714,6.857143,88.142857,36.142857,18.428571,16.428571,84.571429,37.000000,0.0,19.285714,14.714286,1.714286,0.142857,115.857143,59.428571,0.575714,0.428571,5.142857,2.428571,0.601429,161.571429,102.857143,24.571429,14.142857,10.571429,7.857143,98.571429,32.571429,6.428571,4.285714,61.857143,12.428571,0.000000,29.142857,18.142857,1.142857,0.000000,115.571429,44.714286,0.437143,0.285714,3.285714,0.857143,0.147143,158.142857,82.285714,25.0,1062.000000,165.10,167.64,125.0,31.0
2,3,13.866667,8.666667,2.866667,1.733333,116.133333,49.466667,5.333333,4.266667,96.733333,35.600000,0.2,13.733333,11.200000,0.333333,0.133333,124.333333,55.466667,0.430000,1.000000,0.933333,0.400000,0.277333,133.000000,63.400000,14.466667,8.133333,2.800000,0.733333,91.066667,32.200000,4.866667,2.800000,78.266667,23.200000,0.266667,6.000000,4.400000,0.333333,0.133333,98.733333,35.733333,0.340000,0.066667,2.866667,0.666667,0.131333,102.133333,38.600000,33.0,604.400000,180.34,193.04,155.0,35.0
3,3,8.750000,7.500000,3.000000,2.250000,12.750000,4.750000,42.250000,35.750000,44.750000,31.250000,0.0,4.500000,4.000000,7.750000,0.000000,58.000000,42.750000,0.637500,0.500000,5.500000,4.500000,0.817500,101.500000,80.500000,3.000000,2.250000,3.500000,3.000000,5.750000,2.000000,2.000000,1.500000,8.000000,4.000000,0.000000,0.250000,0.250000,0.000000,0.500000,11.250000,6.500000,0.540000,0.750000,0.500000,0.000000,0.000000,38.000000,26.500000,8.0,540.000000,165.10,167.64,115.0,28.0
4,3,25.166667,18.250000,14.000000,11.083333,72.583333,29.416667,16.083333,11.750000,63.000000,23.333333,0.0,14.500000,10.666667,1.500000,0.083333,102.666667,52.250000,0.490000,0.750000,5.333333,1.750000,0.417500,138.333333,85.750000,8.916667,4.666667,5.750000,4.000000,51.000000,12.916667,1.833333,1.166667,43.416667,8.916667,0.250000,6.250000,4.500000,0.833333,0.083333,58.583333,18.083333,0.305833,0.083333,2.250000,1.083333,0.251667,80.583333,37.833333,32.0,750.666667,170.18,180.34,135.0,29.0


# STOP - Modeling Time

## Setup Logging Dictionary

In [4]:
models = ["RANDOM FOREST","XGB GRADIENT BOOSTING"]

#ORDER: 
# MODEL
# PARAMS
# ACCURACY
# FEATRUE IMPORTANCE

log_info = {}

for each in models:
    log_info[each] = []

cols = X.columns

# Random Forest

In [5]:
RFR = RandomForestRegressor(n_estimators=100)
RFR.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [6]:
#RUN AND LOG
RF_Acc = RFR.score(X_test,y_test)
RF_FeatImp = sorted(zip(RFR.feature_importances_.round(4),cols),reverse=True)[:10]
RF_Params = str(RFR.get_params())

print("------RF ACCURACY-------")
print((RF_Acc).round(4))

log_info["RANDOM FOREST"].append(RF_Params)
log_info["RANDOM FOREST"].append(RF_Acc)
log_info["RANDOM FOREST"].append(RF_FeatImp)



------RF ACCURACY-------
0.5749


## XG BOOST

In [7]:
import xgboost as xgb

In [8]:
XGBR = xgb.XGBRegressor(colsample_bytree=0.4,
                gamma=0,
                learning_rate=0.07,
                max_depth=3,
                min_child_weight=1.5,
                n_estimators=1000,
                reg_alpha=0.75,
                reg_lambda=0.45,
                subsample=0.6,
                seed=42)

In [9]:
XGBR.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0, learning_rate=0.07, max_delta_step=0,
       max_depth=3, min_child_weight=1.5, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
       silent=True, subsample=0.6)

In [10]:
XGB_Acc = XGBR.score(X_test,y_test)
XGB_FeatImp = sorted(zip(XGBR.feature_importances_.round(4),cols),reverse=True)[:10]
XGB_Params = str(XGBR.get_params())

print("------XGB ACCURACY-------")
print((XGB_Acc).round(4))

log_info["XGB GRADIENT BOOSTING"].append(XGB_Params)
log_info["XGB GRADIENT BOOSTING"].append(XGB_Acc)
log_info["XGB GRADIENT BOOSTING"].append(XGB_FeatImp)



------XGB ACCURACY-------
0.5979


# LOGGING

In [11]:
log(cols,log_info)

'Logged Successfully'